# Zurich Instruments UHFQA Latency evaluation
Copyright 2019 Zurich Instruments

This script is meant to show the readout time of the UHFQA. On this purpose, a square test signal is generated by the AWG. This is picked up by the Quantum Analyser, integrated, processed and outputted on the trigger lines as result. The integration time is set to the minimum integration time of 4 samples to show the algorithmic delay. The real integration time is dependent on the specific setup and it’s not taken into account here.

To un the test, connect the UHFQA to the computer and set the right device number in the next cell. The Signal Outputs should be conncted to the Signal Inputs with a shotr loop (output one to in one and same for two). An external scope can be used to evaluate the timing. On this purpose it should conncted to
- Output one: the probe signal
- Trigger line one: QA trigger, result ready (Trigger 1)
- Trigger line two: QA unit 1 result

In [1]:
device_uhfqa = 'dev2440'

In [2]:
import textwrap
import time
import numpy as np
import zhinst.utils

# Initialization

In [3]:
daq,_,_ = zhinst.utils.create_api_session(device_uhfqa, 6, 'UHFQA')

Discovered device `dev2440`: UHFQA with options DIG, QC.
Creating an API session for device `dev2440` on `127.0.0.1`, `8004` with apilevel `6`.


# Create and upload AWG program

In [4]:
awg_program = textwrap.dedent("""
    const length = 4096;
    wave w_I = ones(length);
    wave w_Q = ones(length);

    setTrigger(AWG_INTEGRATION_ARM);

    while(true) {
      repeat(100) {
        playWave(w_I, w_Q);
        setTrigger(AWG_INTEGRATION_ARM + AWG_INTEGRATION_TRIGGER + AWG_MONITOR_TRIGGER + 0b10);
        setTrigger(AWG_INTEGRATION_ARM);
        waitWave();
        wait(1024);

        playWave(-w_I, -w_Q);
        setTrigger(AWG_INTEGRATION_ARM + AWG_INTEGRATION_TRIGGER + AWG_MONITOR_TRIGGER + 0b10);
        setTrigger(AWG_INTEGRATION_ARM);
        waitWave();
        wait(1024);
      }
    }
""")

In [5]:
# Create an instance of the AWG module
awgModule = daq.awgModule()
awgModule.set('device', device_uhfqa)
awgModule.set('index', 0)
awgModule.execute()

# Transfer the AWG sequence program. Compilation starts automatically.
awgModule.set('compiler/sourcestring', awg_program)
while awgModule.getInt('compiler/status') == -1:
    time.sleep(0.1)

# Ensure that compilation was successful
assert awgModule.getInt('compiler/status') != 1

# Various setting

In [6]:
daq.setInt(f'/{device_uhfqa}/awgs/0/single', 1)            #No AWG re-run
daq.setDouble(f'/{device_uhfqa}/qas/0/delay', 224)         #Delay of acquisition after AWG wave out
daq.setInt(f'/{device_uhfqa}/qas/0/crosstalk/bypass', 1)   #No crosstalk matrix
daq.setInt(f'/{device_uhfqa}/triggers/out/0/source', 74)   #Trigger out 1 to "QA Result Trigger"
daq.setInt(f'/{device_uhfqa}/triggers/out/1/source', 64)   #Trigger out 2 to "QA Result 1"

integration_length = 4
weights = np.ones(integration_length)
daq.setInt(f'/{device_uhfqa}/qas/0/integration/mode', 0)   #Mode matched integration mode
daq.setInt(f'/{device_uhfqa}/qas/0/integration/length', integration_length) #Integration weights
daq.setVector(f'/{device_uhfqa}/qas/0/integration/weights/0/real', weights)
daq.setVector(f'/{device_uhfqa}/qas/0/integration/weights/0/imag', weights)

daq.sync()

# start measuring

In [7]:
daq.setInt(f'/{device_uhfqa}/sigouts/0/on', 1)
daq.setInt(f'/{device_uhfqa}/sigouts/1/on', 1)
daq.setInt(f'/{device_uhfqa}/triggers/out/0/drive', 1)
daq.setInt(f'/{device_uhfqa}/triggers/out/1/drive', 1)

# Arm the device
daq.asyncSetInt(f'/{device_uhfqa}/awgs/0/single', 1)
daq.syncSetInt(f'/{device_uhfqa}/awgs/0/enable', 1)

1

# Captured scope
- **Green** Probe signal generated by the UHFQA (Output 1)
- **Yellow** QA trigger, result ready (Trigger 1)
- **Orange** QA unit 1 result, high for |1〉, low for |0〉 (Triger 2)

Total delay: 220 ns

![Scope screenshot](scope_screenshot.png)